In [32]:
import cv2, os, numpy as np

In [33]:
target_directory = 'FullDataset' 
output_directory = 'TrainingImages'

In [34]:
os.makedirs(output_directory, exist_ok=True)

In [35]:
def transformar_para_quadrada(imagem, tamanho_final):
    # Obter as dimensões da imagem
    altura, largura = imagem.shape[:2]

    # Calcular o tamanho do quadrado
    lado_quadrado = max(altura, largura)

    # Criar uma nova imagem quadrada com fundo preto
    imagem_quadrada = np.zeros((lado_quadrado, lado_quadrado, 3), dtype=np.uint8)

    # Colocar a imagem original no centro da nova imagem quadrada
    y_offset = (lado_quadrado - altura) // 2
    x_offset = (lado_quadrado - largura) // 2
    
    # Converter a imagem em escala de cinza para 3 canais
    imagem_colorida = cv2.cvtColor(imagem, cv2.COLOR_GRAY2BGR)
    imagem_quadrada[y_offset:y_offset + altura, x_offset:x_offset + largura] = imagem_colorida

    # Redimensionar para o tamanho final desejado
    imagem_quadrada = cv2.resize(imagem_quadrada, (tamanho_final, tamanho_final), interpolation=cv2.INTER_CUBIC)

    return imagem_quadrada

for letter_folder in os.listdir(target_directory):
    letter_path = os.path.join(target_directory, letter_folder)
    
    if os.path.isdir(letter_path):
        for image_file in os.listdir(letter_path):
            image_path = os.path.join(letter_path, image_file)
            
            try:
                image = cv2.imread(image_path)
                if image is not None:
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    imagem_final = transformar_para_quadrada(gray_image, 512)
                    
                    largura_borda = 10  # ajuste conforme necessário
                    imagem_com_borda = cv2.copyMakeBorder(
                        imagem_final,
                        largura_borda,
                        largura_borda,
                        largura_borda,
                        largura_borda,
                        cv2.BORDER_CONSTANT,
                        value=[0, 0, 0]  # cor da borda (preto, por exemplo)
                    )

                    # Garantir que a imagem final não ultrapasse 512x512
                    imagem_com_borda = cv2.resize(imagem_com_borda, (512, 512), interpolation=cv2.INTER_CUBIC)
                                        
                    output_path = os.path.join(output_directory, letter_folder)
                    os.makedirs(output_path, exist_ok=True)
                    
                    gray_image_path = os.path.join(output_path, image_file)
                    cv2.imwrite(gray_image_path, imagem_com_borda)
       
            except Exception as e:
                print(f"Erro ao abrir a imagem {image_path} : {e}")


In [4]:
import cv2
import mediapipe as mp
import pandas as pd
import os

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

# Function to extract hand landmarks
def extract_hand_landmarks(image_path):
    # Load the grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Convert grayscale image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    
    # Process the image to find hand landmarks
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0]
        return [(lm.x, lm.y) for lm in landmarks.landmark]
    return None

# Directory containing ASL images organized by letter
image_directory = 'TrainingImages'
# List to hold data
data = []

# Loop through each subfolder (each letter of the alphabet)
for label in os.listdir(image_directory):
    label_path = os.path.join(image_directory, label)
    
    # Check if it's a directory (skip files in the main folder, if any)
    if os.path.isdir(label_path):
        # Loop through images in each subfolder
        for filename in os.listdir(label_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(label_path, filename)
                landmarks = extract_hand_landmarks(image_path)
                
                if landmarks:
                    # Append the label and landmarks to the data list
                    data.append([label] + [coord for landmark in landmarks for coord in landmark])
    

# Create a DataFrame
columns = ['Label'] + [f'Landmark_{i}_{axis}' for i in range(21) for axis in ['x', 'y']]
df = pd.DataFrame(data, columns=columns)

# Save to CSV
df.to_csv('asl_landmarks.csv', index=False)


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Carregar dados do CSV gerado
df = pd.read_csv('asl_landmarks.csv')

# Separar features (Landmarks) e target (Label)
X = df.drop('Label', axis=1)  # Todos os pontos de landmarks são as features
y = df['Label']               # A coluna Label é o target

# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanciar e treinar o modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = rf_model.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Acurácia: {accuracy}")
print(f"Relatório de Classificação:\n{report}")


Acurácia: 0.9651815799326096
Relatório de Classificação:
              precision    recall  f1-score   support

           A       0.98      0.98      0.98       673
           B       0.98      1.00      0.99       340
           C       0.99      0.95      0.97       124
           D       0.98      0.93      0.95       208
           E       0.93      0.96      0.95       437
           F       0.98      0.99      0.98       336
           G       0.97      1.00      0.98       529
           H       0.98      0.97      0.98       356
           I       0.99      0.97      0.98       248
           J       0.99      0.99      0.99       328
           K       0.99      0.97      0.98       478
           L       0.96      0.97      0.96       267
           M       0.94      0.91      0.92       148
           N       0.94      0.94      0.94       154
           O       0.95      0.91      0.93       115
           P       0.98      0.96      0.97       218
           Q       0.97 